+ Future development plan: 
    -   schedule direct into cobra
    

In [77]:
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
import filepath
import pandas as pd
import glob
import os
import numpy as np
import pygsheets
import infrastructure
import datetime
import math
import datetime
from datetime import timedelta  
import re
import warnings
import statistics
from datetime import date
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import xlsxwriter
import send_email
from colorama import Fore, Style
import matplotlib.pyplot as plt
import seaborn as sns
import filepath 
import datetime as dt
from scipy.stats import percentileofscore
from datetime import datetime
import math 

In [78]:
lanina = infrastructure.get_lanina()

In [79]:
t2=time.time()
sms = pd.read_csv(filepath.output_folder+'SS_LC_merged_data.csv')
df = infrastructure.transform_sms_df(sms)

# df = df.loc[:,~df.columns.duplicated()].copy()

df = df[~df['Send Strategy'].isin(['PT','RT','IT','OT','W','MI','Mi'])]
#testMaster = scheduleMaster[scheduleMaster['Send Strategy'].isin(['PT','RT','IT','OT','CT'])]
#miningMaster = scheduleMaster[scheduleMaster['Send Strategy']=='MI']
#scheduleMaster = scheduleMaster[~(scheduleMaster.index.isin(testMaster.index) | scheduleMaster.index.isin(miningMaster.index))] #create index s time series sorted index
df['Datestring']=df['Date'].dt.strftime('%m.%d.%y') #create datestring variable 
df['CTR']=(df['Clicks']/df['Delivered'])
df['eCPM']=(df['Revenue']*1000/df['Delivered'])
df['JPCTR']=(df['Jump Page Clicks']/df['Delivered'])
df['optout rate']=(df['Optout']/df['Delivered'])
df['CTR Normalized']=(df['CTR']-df['CTR'].min())/(df['CTR'].max()-df['CTR'].min())
df['eCPM Normalized']=(df['eCPM']-df['eCPM'].min())/(df['eCPM'].max()-df['eCPM'].min())
df['CTR50'] = df['CTR Normalized'] + df['eCPM Normalized']
df['Creative Type'] = df['Creative Type'].str.split(' ',expand = True)[0]
# Import Lanina
lanina = infrastructure.get_lanina()
lanina['OfferIDs'] = lanina['OfferIDs'].astype(str).str.split('.',expand = True)[0]
# mamba 
mamba = infrastructure.get_mamba()
    # oppo ctr
temp1= df.groupby(['Segments','Opportunity Cost Send Strategy','Date']).agg({'Revenue':'sum','Clicks':'sum','Delivered':'sum'}).reset_index()
temp1[['rolling Revenue','rolling Clicks','rolling Delivered']] = temp1.groupby('Segments').shift(1).rolling(30, min_periods=5)[['Revenue','Clicks','Delivered']].sum().reset_index(drop=True)
temp1['Dataset_Agg_30day_ctr'] = temp1['rolling Clicks'] / temp1['rolling Delivered']
temp1['Dataset_Agg_30day_ecpm'] = temp1['rolling Revenue'] * 1000/ temp1['rolling Delivered']
dataset_agg_ctr =  temp1[['Segments','Date','Opportunity Cost Send Strategy','Dataset_Agg_30day_ctr','Dataset_Agg_30day_ecpm']]
df = df.merge(dataset_agg_ctr, how = 'left')
df['Opportunity Clicks'] = df['Clicks'] - df['Dataset_Agg_30day_ctr'] * df['Delivered']
df['Opportunity Cost'] = df['Revenue'] - df['Dataset_Agg_30day_ecpm'] * df['Delivered'] /1000  # revenue  - last 30 days average revenue in that drop 
df['opportunity CTR'] = df['Opportunity Clicks'] / df['Delivered']
df['opportunity eCPM'] = df['Opportunity Cost'] * 1000 / df['Delivered']

# change the following code to the function calculate_oppo_ctr
# I need to calculate 
def calculate_oppo_ctr(df):
    df['opportunity CTR'] = df['Opportunity Clicks'] / df['Delivered']
    
    opportunity_ctr_series = df[(df['opportunity CTR']!=np.inf)&(df['opportunity CTR'].isna() == False)]['opportunity CTR']
    #df['opportunity CTR Normalized']=(df['opportunity CTR']-df['opportunity CTR'].min())/(df['opportunity CTR'].max()-df['opportunity CTR'].min())
    df['opportunity CTR Normalized']= percentileofscore(opportunity_ctr_series, df['opportunity CTR'], kind='weak') / 100
    df['opportunity eCPM'] = df['Opportunity Cost'] * 1000 / df['Delivered']
    opportunity_eCPM_series =  df[(df['opportunity eCPM']!=np.inf)&(df['opportunity eCPM'].isna() == False)]['opportunity eCPM']
    #df['opportunity eCPM Normalized']=(df['opportunity eCPM']-df['opportunity eCPM'].min())/(df['opportunity eCPM'].max()-df['opportunity eCPM'].min())
    df['opportunity eCPM Normalized']=percentileofscore(opportunity_eCPM_series, df['opportunity eCPM'], kind='weak') / 100 
    df['opportunity CTR50'] = df['opportunity CTR Normalized']*0.6 + df['opportunity eCPM Normalized']*0.4
    return df

df = calculate_oppo_ctr(df)


# Select by Opportunity Cost CTR50 (60% Oppr CTR Normalized + 40% Oppr eCPM Normalized)

In [123]:

####################### Content Selection with HTML Selection #######################

# ignore all warnings 
import warnings
warnings.filterwarnings('ignore')


def content_select(df,sc,pubid,hitpath,past_content_explode,direct):
    ccid=''
    mamba = infrastructure.get_mamba()
    # according to pubid and sc to find out sc_dppub from df
    publisher = infrastructure.get_publisher()
    dppub = publisher[publisher['PUBID']==int(pubid)]['DP.DS or DP.sV'].values[0]
    dv = publisher[publisher['PUBID']==int(pubid)]['Sub Vertical'].values[0]
    sc_dppub = sc + "_" + dppub
    
    df['Opp Cost eCPM'] = df['Opportunity Cost'] / df['Delivered'] * 1000
    df = df.merge(lanina[['Reporting Content ID','Old Content ID']], how = 'left', left_on = 'Creative Type', right_on = 'Old Content ID')
    df.loc[df['Reporting Content ID'].isna() == False,'Creative Type'] = df['Reporting Content ID']
    df.drop(columns=['Reporting Content ID','Old Content ID'], inplace=True)
    # Create dataframe of all drops of given hitpath in past 90 days
    df_content = df[(df['Date'] > pd.Timestamp.today() - pd.to_timedelta(90, unit='d')) & (df['Hitpath Offer ID']==hitpath) & (df['Shortcode Name']==sc)]
    df_content = df_content[(df_content['Creative Type'].isin(lanina['Reporting Content ID'].unique().tolist())) ]

    
    df_content['shortcode_DP.SV'] = df_content['shortcode_DP.SV'] == sc_dppub 
    df_content['data_vertical'] = df_content['Data Vertical'] == dv
    df_content_by_drop = df_content.copy()
    ## get the opportunity CTR50 for each content piece in the same data vertical 
    df_content_other_affliate = df_content[(df_content['shortcode_DP.SV']==False) & (df_content['data_vertical'] == True)]
    df_content_other_affliate = df_content_other_affliate.groupby(['data_vertical','Creative Type','Hitpath Offer ID'], as_index=False).agg({'Delivered': 'sum', 'Opportunity Cost': 'sum','Opportunity Clicks': 'sum','Clicks': 'sum','Revenue':'sum'})
    df_content_other_affliate = calculate_oppo_ctr(df_content_other_affliate)
    df_content_other_affliate = df_content_other_affliate[['Creative Type','opportunity CTR50']]
    df_content_other_affliate = df_content_other_affliate.rename(columns = {'opportunity CTR50':'opportunity CTR50_data_vertical_level'})


    #offer_median_eCPM = df_content['Opp Cost eCPM'].median()
    offer_median_eCPM = df_content['opportunity CTR50'].median()
    df_content_drops = df_content.groupby(['shortcode_DP.SV','Creative Type'], as_index=False).count()
    df_content_drops = df_content_drops[['shortcode_DP.SV','Creative Type', 'Hitpath Offer ID']]
    df_content_drops.columns = ['shortcode_DP.SV','Creative Type','Drops']
    df_content = df_content.groupby(['shortcode_DP.SV','Creative Type','Hitpath Offer ID'], as_index=False).agg({'Delivered': 'sum', 'Opportunity Cost': 'sum','Opportunity Clicks': 'sum','Clicks': 'sum','Revenue':'sum'})
    df_content = calculate_oppo_ctr(df_content)
    df_content['Calculated Opp Cost eCPM'] = df_content['Opportunity Cost'] / df_content['Delivered'] * 1000
    df_content['Calculated CTR'] = df_content['Clicks'] / df_content['Delivered']
    df_content['Calculated opp CTR'] = df_content['Opportunity Clicks'] / df_content['Delivered']
    #df_content['Calculated Opp Cost eCPM'] = df_content['Opportunity Cost'] / df_content['Delivered'] * 1000
    df_content = df_content.merge(df_content_drops, on=['shortcode_DP.SV','Creative Type'])

    # Get the drops for the affiliate account
    df_content_affliate = df_content[df_content['shortcode_DP.SV']==True]
    df_content_affliate_eCPM = df_content_by_drop[df_content_by_drop['shortcode_DP.SV']==True]['opportunity CTR50'].median()
    df_content_drops = df_content_drops[df_content_drops['shortcode_DP.SV']==True]
    df_content_affliate_drops = df_content_affliate.groupby('Creative Type', as_index=False).count()
    #df_content_affliate_eCPM = df_content_affliate['Opportunity Cost'].sum() / df_content_affliate['Delivered'].sum() * 1000
    df_content_affliate = calculate_oppo_ctr(df_content_affliate)
    #df_content_affliate['Calculated Opp Cost eCPM'] = df_content_affliate['Opportunity Cost'] / df_content_affliate['Delivered'] * 1000
    df_content_affliate = df_content_affliate[['shortcode_DP.SV','Creative Type','opportunity CTR50','Calculated Opp Cost eCPM','Calculated CTR','Calculated opp CTR','Drops']]
    df_content_affliate['Meets Threshold'] =  df_content_affliate['opportunity CTR50'] >= df_content_affliate_eCPM

    # Get date last dropped in affiliate account and whether it is in the past 30 days or not
    df_content_dates = df[(df['Date'] > pd.Timestamp.today() - pd.to_timedelta(90, unit='d')) & \
                        (df['Hitpath Offer ID']==hitpath) & \
                        (df['shortcode_DP.SV'] == sc_dppub)]
    df_content_dates = df_content_dates[['Creative Type','Date']]
    df_content_dates = df_content_dates.groupby(['Creative Type'],as_index=False).max()
    cobra_dates = mamba[(mamba['Hitpath Offer ID']==hitpath) & (mamba['shortcode_DP.SV']==sc_dppub) & (mamba['Creative']!='') & (mamba['Date']> df['Date'].max() ) ][['Date','Creative']]
    cobra_dates['Creative'] = cobra_dates['Creative'].str.replace('NEW','')
    cobra_dates['Creative'] = cobra_dates['Creative'].str.replace('*','')
    cobra_dates['Creative'] = cobra_dates['Creative'].str.split('\n')
    cobra_dates = cobra_dates.explode('Creative')
    cobra_dates = cobra_dates.reset_index(drop=True)
    cobra_dates['Creative'] = cobra_dates['Creative'].str.strip()
    cobra_dates = cobra_dates[cobra_dates['Creative']!='']
    #cobra_dates = mamba[(mamba['Hitpath Offer ID']==hitpath) & (mamba['shortcode_DP.SV']==sc_dppub)][['Date','Creative']]
    cobra_dates.columns = ['Date','Creative Type']
    df_content_dates = pd.concat([df_content_dates,cobra_dates],ignore_index=True).drop_duplicates()
    df_content_dates = df_content_dates.groupby(['Creative Type'],as_index=False).max()
    df_content_dates['Recently Dropped'] = df_content_dates['Date'] > pd.Timestamp.today() - pd.to_timedelta(30, unit='d')

    # Get the drops for all other accounts
    df_content_other = df_content[df_content['shortcode_DP.SV']==False]
    df_content_other_eCPM = df_content_by_drop[df_content_by_drop['shortcode_DP.SV']==False]['opportunity CTR50'].median()
    df_content_drops = df_content_drops[df_content_drops['shortcode_DP.SV']==False]
    #df_content_other_eCPM = df_content_other['Opportunity Cost'].sum() / df_content_other['Delivered'].sum() * 1000
    df_content_other = calculate_oppo_ctr(df_content_other)
    df_content_other = df_content_other.merge(df_content_other_affliate, on = 'Creative Type', how = 'left') 
    df_content_other.loc[df_content_other['opportunity CTR50_data_vertical_level'].isna() == False, 'opportunity CTR50'] = df_content_other['opportunity CTR50_data_vertical_level'] * 0.7 +   df_content_other['opportunity CTR50'] * 0.3
    
    #df_content_other_median_eCPM = df_content_other['Revenue CPM (eCPM)'].median()
    df_content_other = df_content_other[['shortcode_DP.SV','Creative Type','opportunity CTR50','Calculated Opp Cost eCPM','Calculated CTR','Calculated opp CTR','Drops']]
    df_content_other['Meets Threshold'] =  df_content_other['opportunity CTR50'] >= df_content_other_eCPM
  
 


    # Merge the above dataframes together 
    df_content = df_content_other.merge(df_content_affliate, how='outer', on='Creative Type').merge(df_content_dates, how='outer', on='Creative Type')
    df_content['Meets Threshold'] = (df_content['Meets Threshold_y'] == True) | ((df_content['Meets Threshold_y'].isnull()) & (df_content['Meets Threshold_x'] == True))
    print('Offer eCPM for Other Accounts: ', round(df_content_other_eCPM,2))
    print('Offer eCPM for Affiliate Account: ', round(df_content_affliate_eCPM,2))
    print('Offer median eCPM: ',round(offer_median_eCPM,2))

    df_content = df_content[['Creative Type', 'opportunity CTR50_x', 'Drops_x',
                                'opportunity CTR50_y', 'Drops_y', 'Date', 'Recently Dropped', 'Meets Threshold']]
    df_content.columns = ['Creative Type','eCPM in Other Accounts','Drops in Other Accounts',
                            'eCPM in Affiliate Account', 'Drops in Affiliate Accounts','Date','Recently Dropped',
                            'Meets Mean Threshold']
    # check if the content has a median eCPM higher than the offer median eCPM
    df_content_by_drop = df_content_by_drop.groupby(['Creative Type'], as_index=False)['opportunity CTR50'].median()
    df_content = df_content.merge(df_content_by_drop, how='left', on='Creative Type')
    df_content = df_content.rename(columns = {'opportunity CTR50':'median eCPM'})
    df_content['Meets Median Threshold'] = df_content['median eCPM'] >= offer_median_eCPM

    # check if the content has an aggregate eCPM in the affiliate account higher than the offer aggregate eCPM in the affiliate account 
    df_content.loc[df_content['eCPM in Affiliate Account'] >= df_content_affliate_eCPM,'Meets Median Threshold'] = True

    # rank is calculated as the average of the aggregate eCPM of the content piece and two times the median eCPM of the content piece 
    df_content['rank'] = (df_content['eCPM in Other Accounts'] + 2 * df_content['median eCPM']) / 3
    # if the content piece has been dropped in the affiliate account in the past 90 days, that aggregate eCPM will be used in the ranking instead of the aggregate for all accounts
    df_content.loc[~df_content['eCPM in Affiliate Account'].isna(),['rank']] = (df_content['eCPM in Affiliate Account'] + 2*df_content['median eCPM'])/3
    df_content = df_content.sort_values('rank', ascending=False)
    df_content = df_content.merge(lanina[['Reporting Content ID','OfferIDs','Content Approval Status','Content']], how='left', left_on='Creative Type',right_on = 'Reporting Content ID', copy = False )
    # drop reporting content id column
    df_content = df_content.drop(columns=['Reporting Content ID'])
    # filter out content that is not approved and failed testing 
    df_content = df_content[~(df_content['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]
    if direct == True:
        df_content = df_content[df_content['OfferIDs'].astype(int)==int(hitpath)]
    df_content_all = df_content.copy()
    past_content_body = lanina[lanina['Reporting Content ID'].isin(past_content_explode)]['Content'].tolist()
    if df_content.empty:
        
                    # find Reporting Content ID from la nina based on offer IDs and Type 
        lanina1 =  lanina[~(lanina['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]
        df_content= lanina1[(lanina1['OfferIDs'] == hitpath) & (lanina1['Type'] == sc[:3]) & (lanina1['Channel'] !='LC') & (lanina1['Content'].isin(past_content_body)==False)][['Reporting Content ID','Content Approval Status', 'Content']]
        tf_content = len(lanina1[(lanina1['Type'] == sc[:3])]['Content'].unique().tolist())
        z = 2 
        if tf_content >= 14:
            z = 2
        else:
            z = 1
        if len(df_content) > 1: 
            print('No content available from content selection script, we choose 2 random content from Lanina')
            ccid  = df_content.sample(n=z)
            content = ccid['Content'].tolist()
            ccid = ccid['Reporting Content ID'].tolist()
            ccid = "\n".join(ccid)
            df_content_all = df_content.copy()
            print(ccid)
            print(content)
        elif len(df_content) == 1:
            print('No content available from content selection script, we choose 1 random content from Lanina')
            ccid  = df_content['Reporting Content ID'].values[0]
            content = df_content['Content'].values[0]
            df_content_all = df_content.copy()
            print(ccid)
            print(content)
        else:
            
            print('No content available from lanina')
    else:
        # select only content that has been dropped more than 2 times in any account, or has been dropped at least once in the affiliate account it
        df_content['Meets Drop Threshold'] = (df_content['Drops in Other Accounts']>=2) 
        df_content = df_content[((df_content['Meets Drop Threshold']==True) & (df_content['eCPM in Affiliate Account'].isna())) | (~df_content['eCPM in Affiliate Account'].isna())]
        df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

    # if no content meets threshold, remove drop number requirement
        if df_content.empty:
            df_content = df_content_all.copy()
            df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

        # if after remove number of drop restriction no content meets threshold, reduce mean and median threshold until content is found
        k=0
        while df_content.empty:
            k = k + 1
            print('No offers, decreasing thresholds.')
            df_content = df_content_all.copy()
            df_content_other['Meets Threshold'] =  df_content_other['opportunity CTR50'] >= df_content_other_eCPM - k * 0.1
            df_content_affliate['Meets Threshold'] =  df_content_affliate['opportunity CTR50'] >= df_content_affliate_eCPM - k * 0.1
            df_content['Meets Median Threshold'] = df_content['median eCPM'] >= offer_median_eCPM - k * 0.1
            df_content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)].sort_values('rank', ascending=False)

            if k > 20:
                df_content = df_content_all.copy()
                df_content['Meets Drop Threshold'] = (df_content['Drops in Other Accounts'] > 2) 
                df_content = df_content.sort_values('rank', ascending=False)
                break

        df_content = df_content.reset_index(drop=True)
        # If not specifying HTML, remove dates from HTML so gapping is not considered
        
        # Select content, either a random content piece that has not dropped in the past 30 days weighted by the rank or 4 times the rank if opp cost is positive
        # or the one least recently dropped if all have been dropped in the past 30 days
        content = df_content[df_content['Recently Dropped']!=True] # didn't drop in the past 30 days 
        #content.loc[content['rank']>0,'rank'] = content['rank'] * 4 
 
        if content.empty:
            print("All content has been dropped in the past 30 days, selecting the least recently dropped content piece.")
            content = df_content[(df_content['Meets Mean Threshold']==True) & (df_content['Meets Median Threshold']==True)]
            print(len(content))
            selected_creative = content[content['Date'] == content['Date'].min()][['Creative Type']]

            
        else:
            selected_creative = content[['Creative Type']].sample(n=1,weights=pd.to_numeric(content['rank']) + abs(2*min(pd.to_numeric(content['rank']))))
            #selected_creative = content[['Creative Type']]
        try: 
            ccid = selected_creative['Creative Type'].iloc[0]
        except: 
            ccid = df_content_all['Creative Type'].iloc[0]
        print('')
        
        print(ccid)
        print('Ranked', df_content.index[df_content['Creative Type'] == ccid][0] + 1, 'out of', len(df_content), 'pieces that meet thresholds')        

        if len(df_content) == 1:
            print("There's only one content piece that meets thresholds, we try to add a test content")
            lanina1 =  lanina[~(lanina['Content Approval Status'].str.contains('Paused|Not Approved|Failed Testing', na = False)) ]
            lanina_backup_content= lanina1[(lanina1['OfferIDs'] == hitpath) & (lanina1['Type'] == sc[:3]) & (lanina1['Channel'] !='LC') & (lanina1['Content'].isin(past_content_body)==False)&(lanina1['Content Status in Platform']=='Uncreated')][['Reporting Content ID']]
            if len(lanina_backup_content) > 0:
                ccid = ccid + "\n"+lanina_backup_content['Reporting Content ID'].sample(n=1).values[0] 
                print(ccid)
        


    return df_content, df_content_all, ccid 



In [81]:
""" 
passed_content = lanina[lanina['Content Approval Status'].str.contains("Approved - Passed")]
df1 = df[(df['Creative Type'].isin(passed_content['Reporting Content ID'].unique().tolist())) | (df['Hitpath Offer ID']==6272)]
df2 = df1.groupby(['Creative Type'])['Date'].count().reset_index(name = "count")
temp = df2[df2['count'] == 1].merge(lanina[['Reporting Content ID','OfferIDs','Content Approval Status','Offer Status']], left_on = 'Creative Type',right_on = 'Reporting Content ID', how = 'left', copy = False).drop_duplicates().sort_values('OfferIDs', ascending = False).drop(columns = ['Reporting Content ID']).reset_index(drop = True)
temp = temp[temp['Offer Status'] == 'Passed Test - In Production']
temp.to_csv('/Users/liliguo/Desktop/passed_content.csv', index = False) 
"""

' \npassed_content = lanina[lanina[\'Content Approval Status\'].str.contains("Approved - Passed")]\ndf1 = df[(df[\'Creative Type\'].isin(passed_content[\'Reporting Content ID\'].unique().tolist())) | (df[\'Hitpath Offer ID\']==6272)]\ndf2 = df1.groupby([\'Creative Type\'])[\'Date\'].count().reset_index(name = "count")\ntemp = df2[df2[\'count\'] == 1].merge(lanina[[\'Reporting Content ID\',\'OfferIDs\',\'Content Approval Status\',\'Offer Status\']], left_on = \'Creative Type\',right_on = \'Reporting Content ID\', how = \'left\', copy = False).drop_duplicates().sort_values(\'OfferIDs\', ascending = False).drop(columns = [\'Reporting Content ID\']).reset_index(drop = True)\ntemp = temp[temp[\'Offer Status\'] == \'Passed Test - In Production\']\ntemp.to_csv(\'/Users/liliguo/Desktop/passed_content.csv\', index = False) \n'

In [124]:
        
def content_schedule(sc_dppub_affiliate, days_ahead, update_bool = False): 
    lyst = [] 
    
    for i in range(1,math.ceil(days_ahead/7)+1):
        lyst.append(i*7)
    for days_ahead in lyst:  
        begin_day = days_ahead-7
        gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
        cobra_google_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=1186670009')
        cobra_sheet =  cobra_google_sheet.worksheet('title','New Mamba')
        cobra = infrastructure.get_mamba_full_slot()
        cobra_sheet_df = cobra_sheet.get_as_df()
        cobra_sheet_df.columns=cobra_sheet_df.iloc[0]
        
        sc = sc_dppub_affiliate.split('_')[0]
        dppub = sc_dppub_affiliate.split('_')[1]
        pubid = sc_dppub_affiliate.split('_')[2]
        sc_dppub = sc + "_" + dppub
        past_content = cobra[(cobra['Dataset']==sc_dppub_affiliate)& (cobra['Date'].dt.date>=date.today()-timedelta(days=4)) & (cobra['Creative']!='') ][['Date','Creative']]
        past_content['Creative'] = past_content['Creative'].str.split('\n')
        past_content_explode = past_content.explode('Creative')
        content_df=cobra[(cobra['Dataset']==sc_dppub_affiliate)& (cobra['Date'].dt.date>=date.today()+timedelta(days=begin_day)) &
                                    (cobra['Date'].dt.date<=date.today()+timedelta(days=days_ahead))]
                # set row 1 as column 
                #
        
        days=pd.to_datetime(content_df['Date'].unique())

        drops=['Drop 1','Drop 2','Drop 3','Drop 4','Drop 5','Drop 6']
        cols=['Time','Segment ','Send Strategy','Offer','Limit',
                    'Offset','Creative','Job Name','Blank']

        write_dict={}
        
        for index,day in enumerate(pd.to_datetime(content_df['Date'].unique()).strftime('%D')):
            for drop in drops:
                for col in cols:
                    if len(content_df[(content_df["Drop"]==drop) & (content_df['Date']==day)])>0:
                        if col == 'Blank':
                            write_dict[(drop,col)] = ''
                        else: 
                            write_dict[(drop,col)] = content_df[(content_df["Drop"]==drop) & (content_df['Date']==day)][col].values[0]
                        if col == 'Offer' and write_dict[(drop,col)] != '':
                            offer = write_dict[(drop,col)]
                            direct = "DIR" in offer
                            hitpath = write_dict[(drop,col)].split(' ')[0]
                        if col == 'Creative' and write_dict[(drop,col)] == '' and write_dict[(drop,'Offer' )] != '':
                            past_content_list = past_content_explode[(past_content_explode['Date']<pd.to_datetime(day)) & (past_content_explode['Date']>=pd.to_datetime(day)-timedelta(days=4))]['Creative'].tolist()
                            content = content_select(df,sc,pubid,hitpath,past_content_list,direct)[2]
                            write_dict[(drop,col)] = content
                            for i in content.split('\n'):
                                # Convert the new data to a DataFrame
                                new_row = pd.DataFrame([{'Date': pd.to_datetime(day), 'Creative': i}])
                                # Use pd.concat to append the new data
                                past_content_explode = pd.concat([past_content_explode, new_row], ignore_index=True)
                        if col == 'Job Name' and write_dict[(drop,col)] == '' and write_dict[(drop,'Offer' )] != '':
                                date1 = datetime.strptime(day, "%m/%d/%y").strftime("%d%b%y")  
                                #Format: SS_HZB_TLG-PL-30DC-VZN_12386_P_21Oct23    
                                write_dict[(drop,col)] =   "SS_"+write_dict[(drop,'Segment ')][:3] + "_"+write_dict[(drop,'Segment ')][4:].replace(".",'-').replace("_",'-')+"_"+ write_dict[(drop,'Offer')].split(' ')[0] +"_"+ write_dict[(drop,'Send Strategy')] + "_" +   date1       

                    else:
                        write_dict[(drop,col)] = ''

            if index==0:
                write_df = pd.DataFrame(write_dict,index=[day]).T
                write_df1 = pd.DataFrame(write_dict,index=[day]).T

            else:
                write_df_new = pd.DataFrame(write_dict,index=[day]).T
                write_df = pd.concat([write_df,write_df_new],axis=1)

            #write_df=write_df.replace(np.nan,'')
        if update_bool == True:
                    
            cobra_row = int(cobra_sheet_df[cobra_sheet_df['Dataset'].str.contains(sc_dppub_affiliate)].index.values[0]+3)
            if len(content_df) >0:
                date_string = content_df['Date'].dt.date.values[0].strftime('%-m/%-d/%Y')
                #3.11.5 python version accept this: date_string = content_df['Date'].dt.date.values[0].strftime('%#m/%#d/%Y')
                cobra_column = cobra_sheet_df.columns.get_loc(date_string)+1
                cobra_sheet.set_dataframe(start=(cobra_row,cobra_column),df=write_df,copy_head=False,nan='')
                
        else: 
            return write_df
        


In [125]:
#sc_dppub_affiliates = ['HZB_CM.OSR_461452','MBC_CM.OSR_461452','SVT_AMD.PL_461810','HZB_AMD.PL_461810','SVT_AL.PL.3_461838','HZB_AL.PL.3_461838','DSS_TLG.PL_461768','HZB_TLG.PL_461768','MBC_WWM.YFA.2_461680','FLC_WWM.YFA.2_461680','DSS_SM.SRV_461896','HZB_SM.SRV_461896'] #the sc_dppub_affiliate for which to schedule (list of strings)
sc_dppub_affiliates =  [
    "FLC_I.CC_460918", "MBC_I.CC_460918", "FLC_SPK.CR_460921",
    "MBC_SPK.CR_460921", "FLC_AI.CC_460939", "MBC_AI.CC_460939", "HZB_RHD.CC_461263",
    "DSS_RHD.CC_461263","UAA_SPK.CR2_461842","UAATF_SPK.CR2_461842","MBC_PN.FC_461653","FLC_PN.FC_461653",
    "SVT_AL.PL_461794","HZB_AL.PL_461794","SVT_AL.PL.2_461795","HZB_AL.PL.2_461795",
    "DSS_JET.ZTA_461835","FLC_JET.ZTA_461835", "FLC_SPK.CR2_461842","MBC_PN.SWP_461500", "FLC_PN.SWP_461500", 'FLC_EDM.247L_461227','MBC_EDM.247L_461227']
# sc_dppub_affiliates = ['SVT_AL.PL.2_461795']
#sc_dppub_affiliates = ['MFA_I.MFA_461871']
days_ahead = 20
update_bool = True 

for sc_dppub_affiliate in sc_dppub_affiliates: 
    try:
        content_schedule(sc_dppub_affiliate, days_ahead, update_bool)
    except Exception as e:
        print(f"An error occurred with {sc_dppub_affiliate}: {e}")


Offer eCPM for Other Accounts:  nan
Offer eCPM for Affiliate Account:  nan
Offer median eCPM:  nan
No content available from content selection script, we choose 2 random content from Lanina
13238.TF.MFA.454341
13238.TF.MFA.454343
['MyFam: Want to pay less for household repairs, {{coalesce|first_name|starting now}}?\n\nCheck out these offers: {{offer_target_url}}\n\nSTOP to end', 'MyFam: {{coalesce|first_name|Congrats}}, we found home warranty plans in {{coalesce|City|your area}}.\n\nView here: {{offer_target_url}}\n\nSTOP to end']


# Check recent content body

In [92]:
# show all content from the column 
pd.set_option('display.max_colwidth', None)
sc_dppub_affiliate = 'MFA_I.MFA_461871'
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
cobra_google_sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/12vqSDueybprNphtsw7gXR5vmgcPG6_5ZNcnWzNpiasY/edit#gid=1186670009')
cobra_sheet =  cobra_google_sheet.worksheet('title','New Mamba')
cobra = infrastructure.get_mamba_full_slot()
past_content = cobra[(cobra['Dataset']==sc_dppub_affiliate)& (cobra['Date'].dt.date>=date.today()-timedelta(days=4)) & (cobra['Creative']!='') ][['Date','Creative']]
past_content['Creative'] = past_content['Creative'].str.split('\n')
past_content_explode = past_content.explode('Creative')
past_content_explode.merge(lanina[['Reporting Content ID','Content Approval Status','Content']], how = 'left', left_on = 'Creative', right_on = 'Reporting Content ID', copy = False)

,Date,Creative,Reporting Content ID,Content Approval Status,Content
0,2024-05-20,102.TF.MFA.453992,102.TF.MFA.453992,Paused - Others,"MyFam: {{coalesce|first_name|Congrats}}, a credit card offer is available as of {{today_mm_dd_yy}}.\n\nView here: {{offer_target_url}}\n\nSTOP to end"
1,2024-05-21,103.TF.MFA.454012,103.TF.MFA.454012,Live,MyFam: You’re eligible on {{today_mm_dd_yy}} to request financial relief.\n\nRequest an e-deposit now: {{offer_target_url}}\n\nSTOP to end
2,2024-05-22,TF.MFA.453582,TF.MFA.453582,Live,MyFam: Here is the guide you requested! Access now: {{offer_target_url}}\n\nSTOP to end
3,2024-05-23,104.SC.MFA.454212,104.SC.MFA.454212,Live,MyFam: Review your {{today_month}} home coverage offer on {{today_mm_dd_yy}} before it expires: {{offer_target_url}}\n\nSTOP to end
4,2024-05-24,103.TF.MFA.454012,103.TF.MFA.454012,Live,MyFam: You’re eligible on {{today_mm_dd_yy}} to request financial relief.\n\nRequest an e-deposit now: {{offer_target_url}}\n\nSTOP to end
5,2024-05-27,13194.TF.MFA.454343,13194.TF.MFA.454343,Live,"MyFam: {{max_length|14||first_name|Hey}}, we processed your e-deposit request on {{today_mm_dd_yy}}.\n\nGet instant relief today: {{offer_target_url}}\n\nSTOP to end"
6,2024-05-27,13194.TF.MFA.454345,13194.TF.MFA.454345,Live,MyFam: Check your family’s newest relief opportunities as of {{today_mm_dd_yy}}: {{offer_target_url}}\n\nSTOP to end
7,2024-05-29,12972.TF.MFA.454271,12972.TF.MFA.454271,Live,MyFam: You received a grant request for free federal relief on {{today_mm_dd_yy}}.\n\nApply now: {{offer_target_url}}\n\nSTOP to end
8,2024-05-30,13253.TF.MFA.454252,13253.TF.MFA.454252,Live,"MyFam: Access granted - open your credit card options, {{coalesce|first_name|instantly}}.\n\nIt takes just 30 seconds: {{offer_target_url}}\n\nSTOP to end"
9,2024-05-30,13253.TF.MFA.454251,13253.TF.MFA.454251,Live,"MyFam: You’re invited on {{today_mm_dd_yy}} to access card offers of up to 5,000.\n\nGet Started: {{offer_target_url}}\n\nSTOP to end"
